# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.


### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:**
I started with the same architecture and variables as in the [paper](https://arxiv.org/pdf/1411.4555).
"We used 512 dimensions for the embeddings and the size of the LSTM memory."


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:**
I left the transform as it was. The transform is a good choice because it resizes the image to 256x256 and then crops it to 224x224. This is a common practice in image processing and it is used in many models. The transform also normalizes the image for the pre-trained model.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:**
I selected all of the parameters in the decoder and the embedding layer of the encoder. The decoder needs to learn the weights of the LSTM layers and the embedding layer of the encoder needs to learn the weights of the embeddings. The other layers in the encoder are pre-trained and do not need to be trained. 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:**
In terms of the optimizer, I chose the Adam optimizer because it is a popular optimizer and it is used in many models. I used a step shceduler to decrease the learning rate by a factor of 0.2 every epoch as the loss seemed to plateau after about one epoch.

In [1]:
!pip install tensorboard

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 14.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.6 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 33.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
%matplotlib inline

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import sys
sys.path.append('/mnt/disks/legacy-jupytergpu-data/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 4       # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512          # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
save_every = 1             # determines frequency of saving model weights
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 128
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader (TODO: You can change the subset_size, however a 
# very large value would increase training time considerably)
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file,
                         cocoapi_loc='/mnt/disks/legacy-jupytergpu-data/', 
                         subset_size=16000
                        )





Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.06s)
creating index...
index created!
Obtaining caption lengths for the subset...


100%|██████████| 16000/16000 [00:02<00:00, 5880.22it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [10]:
num_epochs = 3          # number of training epochs
starting_point = 1

In [14]:
# The size of the vocabulary.
import os
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=1)

# or... load from previous training and try to improve
encoder_file = 'encoder-5.pkl'
decoder_file = 'decoder-5.pkl'
load_models = False
if load_models:
    encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
    decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
print_every = 1          # determines window for printing average loss

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.01, betas=(0.9, 0.99), eps=1e-08)
scheduler = StepLR(optimizer, step_size=1, gamma=0.2)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

In [15]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time


# Open the training log file.
f = open(log_file, 'w')

accumulation_steps = 1  # Define how many steps to accumulate gradients
optimizer.zero_grad()   # Initialize gradients to zero

for epoch in range(1, num_epochs+1):

    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
                
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        loss = loss / accumulation_steps  # Scale the loss by the accumulation steps


        # Backward pass.
        loss.backward()
        
        if i_step % accumulation_steps == 0 or i_step == total_step:
            # Update the parameters in the optimizer and zero the gradients.
            optimizer.step()
            optimizer.zero_grad()
                    
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item() * accumulation_steps, np.exp(loss.item() * accumulation_steps))
        
        # Print training statistics (on same line).

        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            print(f" Current Learning Rate: {optimizer.param_groups[0]['lr']}")
            # Append current batch loss
            # Track loss and step number        

        
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % (epoch + starting_point)))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % (epoch + starting_point)))
    scheduler.step()

# Close the training log file.
f.close()


Epoch [1/3], Step [1/250], Loss: 9.2026, Perplexity: 9922.6065
 Current Learning Rate: 0.01
Epoch [1/3], Step [2/250], Loss: 8.5066, Perplexity: 4947.1834
 Current Learning Rate: 0.01
Epoch [1/3], Step [3/250], Loss: 5.5339, Perplexity: 253.1395
 Current Learning Rate: 0.01
Epoch [1/3], Step [4/250], Loss: 5.3033, Perplexity: 201.0046
 Current Learning Rate: 0.01
Epoch [1/3], Step [5/250], Loss: 5.5344, Perplexity: 253.2569
 Current Learning Rate: 0.01
Epoch [1/3], Step [6/250], Loss: 4.8605, Perplexity: 129.0930
 Current Learning Rate: 0.01
Epoch [1/3], Step [7/250], Loss: 4.8063, Perplexity: 122.2805
 Current Learning Rate: 0.01
Epoch [1/3], Step [8/250], Loss: 4.9440, Perplexity: 140.3310
 Current Learning Rate: 0.01
Epoch [1/3], Step [9/250], Loss: 4.4293, Perplexity: 83.8693
 Current Learning Rate: 0.01
Epoch [1/3], Step [10/250], Loss: 4.3995, Perplexity: 81.4080
 Current Learning Rate: 0.01
Epoch [1/3], Step [11/250], Loss: 4.3155, Perplexity: 74.8489
 Current Learning Rate: 0.0

Epoch [1/3], Step [92/250], Loss: 3.1106, Perplexity: 22.4342
 Current Learning Rate: 0.01
Epoch [1/3], Step [93/250], Loss: 3.3302, Perplexity: 27.9426
 Current Learning Rate: 0.01
Epoch [1/3], Step [94/250], Loss: 3.2252, Perplexity: 25.1589
 Current Learning Rate: 0.01
Epoch [1/3], Step [95/250], Loss: 3.0458, Perplexity: 21.0268
 Current Learning Rate: 0.01
Epoch [1/3], Step [96/250], Loss: 3.0599, Perplexity: 21.3261
 Current Learning Rate: 0.01
Epoch [1/3], Step [97/250], Loss: 3.1537, Perplexity: 23.4234
 Current Learning Rate: 0.01
Epoch [1/3], Step [98/250], Loss: 3.1743, Perplexity: 23.9089
 Current Learning Rate: 0.01
Epoch [1/3], Step [99/250], Loss: 4.0709, Perplexity: 58.6084
 Current Learning Rate: 0.01
Epoch [1/3], Step [100/250], Loss: 2.9288, Perplexity: 18.7059
 Current Learning Rate: 0.01
Epoch [1/3], Step [101/250], Loss: 3.2676, Perplexity: 26.2472
 Current Learning Rate: 0.01
Epoch [1/3], Step [102/250], Loss: 3.6992, Perplexity: 40.4166
 Current Learning Rate: 0

Epoch [1/3], Step [182/250], Loss: 3.2028, Perplexity: 24.6008
 Current Learning Rate: 0.01
Epoch [1/3], Step [183/250], Loss: 3.0650, Perplexity: 21.4350
 Current Learning Rate: 0.01
Epoch [1/3], Step [184/250], Loss: 3.0422, Perplexity: 20.9505
 Current Learning Rate: 0.01
Epoch [1/3], Step [185/250], Loss: 3.1365, Perplexity: 23.0232
 Current Learning Rate: 0.01
Epoch [1/3], Step [186/250], Loss: 2.8899, Perplexity: 17.9916
 Current Learning Rate: 0.01
Epoch [1/3], Step [187/250], Loss: 2.8261, Perplexity: 16.8791
 Current Learning Rate: 0.01
Epoch [1/3], Step [188/250], Loss: 2.8081, Perplexity: 16.5784
 Current Learning Rate: 0.01
Epoch [1/3], Step [189/250], Loss: 2.9428, Perplexity: 18.9681
 Current Learning Rate: 0.01
Epoch [1/3], Step [190/250], Loss: 2.7665, Perplexity: 15.9036
 Current Learning Rate: 0.01
Epoch [1/3], Step [191/250], Loss: 2.9910, Perplexity: 19.9048
 Current Learning Rate: 0.01
Epoch [1/3], Step [192/250], Loss: 3.1209, Perplexity: 22.6665
 Current Learning

Epoch [2/3], Step [22/250], Loss: 2.6037, Perplexity: 13.5137
 Current Learning Rate: 0.002
Epoch [2/3], Step [23/250], Loss: 2.8861, Perplexity: 17.9232
 Current Learning Rate: 0.002
Epoch [2/3], Step [24/250], Loss: 2.8563, Perplexity: 17.3972
 Current Learning Rate: 0.002
Epoch [2/3], Step [25/250], Loss: 3.2894, Perplexity: 26.8261
 Current Learning Rate: 0.002
Epoch [2/3], Step [26/250], Loss: 2.8302, Perplexity: 16.9484
 Current Learning Rate: 0.002
Epoch [2/3], Step [27/250], Loss: 2.6649, Perplexity: 14.3664
 Current Learning Rate: 0.002
Epoch [2/3], Step [28/250], Loss: 2.6392, Perplexity: 14.0018
 Current Learning Rate: 0.002
Epoch [2/3], Step [29/250], Loss: 2.6183, Perplexity: 13.7129
 Current Learning Rate: 0.002
Epoch [2/3], Step [30/250], Loss: 2.6327, Perplexity: 13.9119
 Current Learning Rate: 0.002
Epoch [2/3], Step [31/250], Loss: 2.5541, Perplexity: 12.8600
 Current Learning Rate: 0.002
Epoch [2/3], Step [32/250], Loss: 2.6454, Perplexity: 14.0891
 Current Learning 

Epoch [2/3], Step [111/250], Loss: 2.4759, Perplexity: 11.8920
 Current Learning Rate: 0.002
Epoch [2/3], Step [112/250], Loss: 2.3337, Perplexity: 10.3156
 Current Learning Rate: 0.002
Epoch [2/3], Step [113/250], Loss: 2.4835, Perplexity: 11.9831
 Current Learning Rate: 0.002
Epoch [2/3], Step [114/250], Loss: 2.4142, Perplexity: 11.1813
 Current Learning Rate: 0.002
Epoch [2/3], Step [115/250], Loss: 2.6257, Perplexity: 13.8140
 Current Learning Rate: 0.002
Epoch [2/3], Step [116/250], Loss: 2.2291, Perplexity: 9.2912
 Current Learning Rate: 0.002
Epoch [2/3], Step [117/250], Loss: 2.4562, Perplexity: 11.6608
 Current Learning Rate: 0.002
Epoch [2/3], Step [118/250], Loss: 2.5693, Perplexity: 13.0571
 Current Learning Rate: 0.002
Epoch [2/3], Step [119/250], Loss: 2.6994, Perplexity: 14.8713
 Current Learning Rate: 0.002
Epoch [2/3], Step [120/250], Loss: 2.6668, Perplexity: 14.3935
 Current Learning Rate: 0.002
Epoch [2/3], Step [121/250], Loss: 2.5934, Perplexity: 13.3749
 Current

Epoch [2/3], Step [200/250], Loss: 2.2242, Perplexity: 9.2457
 Current Learning Rate: 0.002
Epoch [2/3], Step [201/250], Loss: 2.3041, Perplexity: 10.0148
 Current Learning Rate: 0.002
Epoch [2/3], Step [202/250], Loss: 2.4561, Perplexity: 11.6588
 Current Learning Rate: 0.002
Epoch [2/3], Step [203/250], Loss: 2.2940, Perplexity: 9.9145
 Current Learning Rate: 0.002
Epoch [2/3], Step [204/250], Loss: 2.4254, Perplexity: 11.3068
 Current Learning Rate: 0.002
Epoch [2/3], Step [205/250], Loss: 2.2023, Perplexity: 9.0458
 Current Learning Rate: 0.002
Epoch [2/3], Step [206/250], Loss: 2.3686, Perplexity: 10.6828
 Current Learning Rate: 0.002
Epoch [2/3], Step [207/250], Loss: 2.3303, Perplexity: 10.2812
 Current Learning Rate: 0.002
Epoch [2/3], Step [208/250], Loss: 2.8235, Perplexity: 16.8362
 Current Learning Rate: 0.002
Epoch [2/3], Step [209/250], Loss: 2.3076, Perplexity: 10.0499
 Current Learning Rate: 0.002
Epoch [2/3], Step [210/250], Loss: 2.3451, Perplexity: 10.4339
 Current L

Epoch [3/3], Step [39/250], Loss: 2.1691, Perplexity: 8.7502
 Current Learning Rate: 0.0004
Epoch [3/3], Step [40/250], Loss: 2.2271, Perplexity: 9.2726
 Current Learning Rate: 0.0004
Epoch [3/3], Step [41/250], Loss: 2.3568, Perplexity: 10.5570
 Current Learning Rate: 0.0004
Epoch [3/3], Step [42/250], Loss: 2.1416, Perplexity: 8.5130
 Current Learning Rate: 0.0004
Epoch [3/3], Step [43/250], Loss: 2.2980, Perplexity: 9.9547
 Current Learning Rate: 0.0004
Epoch [3/3], Step [44/250], Loss: 2.1904, Perplexity: 8.9388
 Current Learning Rate: 0.0004
Epoch [3/3], Step [45/250], Loss: 2.4076, Perplexity: 11.1074
 Current Learning Rate: 0.0004
Epoch [3/3], Step [46/250], Loss: 2.1863, Perplexity: 8.9022
 Current Learning Rate: 0.0004
Epoch [3/3], Step [47/250], Loss: 2.7427, Perplexity: 15.5291
 Current Learning Rate: 0.0004
Epoch [3/3], Step [48/250], Loss: 2.3399, Perplexity: 10.3803
 Current Learning Rate: 0.0004
Epoch [3/3], Step [49/250], Loss: 2.4005, Perplexity: 11.0291
 Current Learn

Epoch [3/3], Step [128/250], Loss: 2.2022, Perplexity: 9.0445
 Current Learning Rate: 0.0004
Epoch [3/3], Step [129/250], Loss: 2.3233, Perplexity: 10.2098
 Current Learning Rate: 0.0004
Epoch [3/3], Step [130/250], Loss: 2.2105, Perplexity: 9.1199
 Current Learning Rate: 0.0004
Epoch [3/3], Step [131/250], Loss: 2.2850, Perplexity: 9.8256
 Current Learning Rate: 0.0004
Epoch [3/3], Step [132/250], Loss: 2.4778, Perplexity: 11.9148
 Current Learning Rate: 0.0004
Epoch [3/3], Step [133/250], Loss: 2.1335, Perplexity: 8.4441
 Current Learning Rate: 0.0004
Epoch [3/3], Step [134/250], Loss: 2.3604, Perplexity: 10.5957
 Current Learning Rate: 0.0004
Epoch [3/3], Step [135/250], Loss: 2.6407, Perplexity: 14.0234
 Current Learning Rate: 0.0004
Epoch [3/3], Step [136/250], Loss: 2.4331, Perplexity: 11.3936
 Current Learning Rate: 0.0004
Epoch [3/3], Step [137/250], Loss: 2.0785, Perplexity: 7.9923
 Current Learning Rate: 0.0004
Epoch [3/3], Step [138/250], Loss: 2.2081, Perplexity: 9.0989
 Cu

Epoch [3/3], Step [216/250], Loss: 2.0097, Perplexity: 7.4608
 Current Learning Rate: 0.0004
Epoch [3/3], Step [217/250], Loss: 2.4350, Perplexity: 11.4163
 Current Learning Rate: 0.0004
Epoch [3/3], Step [218/250], Loss: 2.1500, Perplexity: 8.5848
 Current Learning Rate: 0.0004
Epoch [3/3], Step [219/250], Loss: 2.1917, Perplexity: 8.9506
 Current Learning Rate: 0.0004
Epoch [3/3], Step [220/250], Loss: 2.2399, Perplexity: 9.3923
 Current Learning Rate: 0.0004
Epoch [3/3], Step [221/250], Loss: 2.1608, Perplexity: 8.6778
 Current Learning Rate: 0.0004
Epoch [3/3], Step [222/250], Loss: 2.1765, Perplexity: 8.8151
 Current Learning Rate: 0.0004
Epoch [3/3], Step [223/250], Loss: 3.2868, Perplexity: 26.7584
 Current Learning Rate: 0.0004
Epoch [3/3], Step [224/250], Loss: 2.2754, Perplexity: 9.7315
 Current Learning Rate: 0.0004
Epoch [3/3], Step [225/250], Loss: 2.3687, Perplexity: 10.6831
 Current Learning Rate: 0.0004
Epoch [3/3], Step [226/250], Loss: 2.1876, Perplexity: 8.9135
 Curr

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [16]:
# (Optional) TODO: Validate your model.